In [1]:
## imports
from dotenv import load_dotenv
from langchain.agents import create_agent
from src.instructions.seo_agent_instruction import seo_agent_instruction
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
import os
from langchain_core.messages import HumanMessage

In [2]:
load_dotenv(override=True)

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
# os.environ["DATAFORSEO_USERNAME"] = "hitesh.solanki@strique.io"
# os.environ["DATAFORSEO_PASSWORD"] = "7836de8842bcdfc8"

In [3]:
class SEOAgent:
    def __init__(self, llm: ChatOpenAI):
        self.name = "SEO Agent"
        self.description = seo_agent_instruction()
        self.model = llm
        
    def get_mcp_client(self):
        return MultiServerMCPClient(
            {
                "gscServer": {
                    "command": "/Users/Hemant/Desktop/strique/mcp-gsc/.venv/bin/python",
                    "args": ["/Users/Hemant/Desktop/strique/mcp-gsc/gsc_server.py"],
                    "transport": "stdio",
                    "env": {
                        "GSC_CREDENTIALS_PATH": "/Users/Hemant/Downloads/service_account_credentials.json",
                        "GSC_SKIP_OAUTH": "true"
                    }
                },
                "dataforseo": {
                    "transport": "streamable_http",
                    "url": "https://dataforseo-mcp-worker.hitesh-solanki.workers.dev/mcp"
                }
            }
        )
    
    async def get_tools(self):
        # Initialize the MCP client
        client = self.get_mcp_client()
        
        # Get the tools
        self.tools = await client.get_tools()
        return self.tools

    async def get_agent(self):
        # Get the tools
        tools = await self.get_tools()
        
        # Create the agent
        self.agent = create_agent(
            model=self.model,
            tools=tools,
            system_prompt=self.description
        )
        
        return self.agent
        
    async def run(self, messages):
        # Get the agent
        agent = await self.get_agent()
            
        return await agent.ainvoke({
            "messages": messages
        })

In [4]:
# Initialize the LLM
llm = ChatOpenAI(model="gpt-5-mini-2025-08-07", temperature=0)

seo_agent = SEOAgent(llm)

In [5]:
tools = await seo_agent.get_tools()

In [6]:
tools

[StructuredTool(name='list_properties', description="\n    Retrieves and returns the user's Search Console properties.\n    ", args_schema={'properties': {}, 'title': 'list_propertiesArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1114639c0>),
 StructuredTool(name='add_site', description='\n    Add a site to your Search Console properties.\n    \n    Args:\n        site_url: The URL of the site to add (must be exact match e.g. https://example.com, or https://www.example.com, or https://subdomain.example.com/path/, for domain properties use format: sc-domain:example.com)\n    ', args_schema={'properties': {'site_url': {'title': 'Site Url', 'type': 'string'}}, 'required': ['site_url'], 'title': 'add_siteArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x10bf1e8e0>),
 StructuredTool(name='de

In [9]:
# add the tools name and description to the csv file
import pandas as pd

# First, let's inspect the structure of tools
print(f"Type of tools: {type(tools)}")
print(f"Length: {len(tools) if isinstance(tools, list) else 'N/A'}")
if isinstance(tools, list) and len(tools) > 0:
    print(f"First tool type: {type(tools[0])}")
    print(f"First tool: {tools[0]}")
    print(f"First tool attributes: {dir(tools[0]) if hasattr(tools[0], '__dict__') else 'N/A'}")

# Extract name and description from tools
tools_data = []
for tool in tools:
    # Handle different possible structures
    if hasattr(tool, 'name'):
        name = tool.name
    elif isinstance(tool, dict) and 'name' in tool:
        name = tool['name']
    else:
        name = str(tool)
    
    if hasattr(tool, 'description'):
        description = tool.description
    elif isinstance(tool, dict) and 'description' in tool:
        description = tool['description']
    else:
        description = ''
    
    tools_data.append({'name': name, 'description': description})

tools_df = pd.DataFrame(tools_data)
tools_df.to_csv('tools.csv', index=False)
print(f"\nCreated DataFrame with {len(tools_df)} tools")
print(tools_df.head())



Type of tools: <class 'list'>
Length: 86
First tool type: <class 'langchain_core.tools.structured.StructuredTool'>
First tool: name='list_properties' description="\n    Retrieves and returns the user's Search Console properties.\n    " args_schema={'properties': {}, 'title': 'list_propertiesArguments', 'type': 'object'} response_format='content_and_artifact' coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1114639c0>
First tool attributes: ['InputType', 'OutputType', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__firstlineno__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or

In [22]:
## Use Human Message
messages = [
    HumanMessage(content="Get the top 5 combined results (ads + organic) for iphone 16 price' on Google India, mobile. Include {rank, type: 'ad'|'organic', title, url, domain}")
]
# messages = [
#     HumanMessage(content="List all properties.")
# ]

result = await seo_agent.run(messages)
print(result['messages'][-1].content)

[
  {"rank": 3, "type": "organic", "title": "Buy iPhone 16", "url": "https://www.apple.com/in/shop/buy-iphone/iphone-16", "domain": "www.apple.com"},
  {"rank": 4, "type": "organic", "title": "iPhone 16 - IWIT (White, 128 GB)", "url": "https://www.flipkart.com/apple-iphone-16-white-128-gb/p/itm09e9f0a0f5ca0", "domain": "www.flipkart.com"},
  {"rank": 7, "type": "organic", "title": "iPhone - Apple (IN)", "url": "https://www.apple.com/in/iphone/", "domain": "www.apple.com"},
  {"rank": 8, "type": "organic", "title": "iPhone 16 128 GB: 5G Mobile Phone with Camera Control, A18 Chip and a ...", "url": "https://www.amazon.in/iPhone-16-128-GB-Control/dp/B0DGHZWBYB", "domain": "www.amazon.in"},
  {"rank": 11, "type": "organic", "title": "Apple iPhone 16: Price in India, Features, and Specifications", "url": "https://www.bajajfinserv.in/iphone-16-price-in-india", "domain": "www.bajajfinserv.in"}
]


In [35]:
tools = await seo_agent.get_tools()